In [ ]:
from pandas import read_csv, DataFrame
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import seaborn as sns
sns.set_theme()

In [ ]:
plt.rcParams.update({
        'font.family': 'serif',
        'font.serif': [],
        'font.size' : 11,
        'text.usetex': True,
        'pgf.texsystem': 'pdflatex',
        'axes.facecolor':'white',
        'axes.edgecolor': 'lightgrey'
        })

MEDIUM_SIZE = 14
BIGGER_SIZE = 16
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize

In [ ]:
INPUT = "-BBB"
FRAMESIZE = 60
#TAS_CONFIG = 1
STREAMS = ["robotic", "audio", "video"]

configs = ['FibroLAN-FalconRX{}_No-QoS_{}_0.0percent'.format(INPUT, FRAMESIZE),
           'FibroLAN-FalconRX{}_No-QoS_{}_120.0percent'.format(INPUT, FRAMESIZE),
           'FibroLAN-FalconRX{}_IEEE-802.1p_{}_120.0percent'.format(INPUT, FRAMESIZE),
           'FibroLAN-FalconRX{}_TAS{}_{}_120.0percent'.format(INPUT, 1, FRAMESIZE),
           'FibroLAN-FalconRX{}_TAS{}_{}_120.0percent'.format(INPUT, 5, FRAMESIZE)]
rel_path = '../measurements/successful_measurements/'

dfs = {}
for c in configs:
    dfs[c] = {}
    for s in STREAMS:
        dfs[c][s] = read_csv('{}{}/csv/{}.csv'.format(rel_path, c, s),
                             delimiter=",",
                             usecols=['Latency (ns)'],
                             dtype=int)

In [ ]:
pkt_count=[]
stats_dfs = []

for df in dfs[configs[3]].values():
    stats_df = df.groupby('Latency (ns)')['Latency (ns)'].agg('count').pipe(DataFrame) \
                .rename(columns = {'Latency (ns)': 'frequency'})
    stats_df['pmf'] = stats_df['frequency'] / sum(stats_df['frequency'])
    pkt_count.append(sum(stats_df['frequency']))
    stats_df['cdf'] = stats_df['pmf'].cumsum()
    stats_df['ccdf'] = 1 - stats_df['pmf'].cumsum()
    stats_df.iloc[-1, stats_df.columns.get_loc('ccdf')] = 0
    stats_df = stats_df.reset_index()
    stats_dfs.append(stats_df)

plt.plot(stats_dfs[0]['Latency (ns)'], stats_dfs[0]['ccdf'], '--')
plt.plot(stats_dfs[1]['Latency (ns)'], stats_dfs[1]['ccdf'], '-.')
plt.plot(stats_dfs[2]['Latency (ns)'], stats_dfs[2]['ccdf'], ':')

colors = [c for c in sns.color_palette()]
"""
robotic_legend = mpatches.Patch(color=colors[0],
                                label='Tactile')
audio_legend = mpatches.Patch(color=colors[1],
                              label='Audio')
video_legend = mpatches.Patch(color=colors[2],
                              label='Video')
"""
robotic_legend = mlines.Line2D([], [], color=colors[0], linestyle='--',
                          markersize=15, label='Tactile')
audio_legend = mlines.Line2D([], [], color=colors[1], linestyle='-.',
                          markersize=15, label='Audio')
video_legend = mlines.Line2D([], [], color=colors[2], linestyle=':',
                          markersize=15, label='Video')
plt.legend(handles=[robotic_legend,
                    audio_legend,
                    video_legend],
           loc='upper right')

plt.grid(visible=True,
         which='major',
         color='lightgrey',
         linestyle='-')
plt.grid(visible=True,
         which='minor',
         color='lightgrey',
         linestyle='--')

plt.yscale("log")
plt.xscale("log")
plt.ylim((3e-5,1.3))
plt.xlim((3e3,3e6))

plt.xlabel("Latency [ns]")
plt.ylabel("CCDF")

plt.tight_layout()
plt.savefig("ccdfplot-TAS1_bg.pdf", dpi=300)
plt.close("all")